# 네이버 뉴스 기사 건수 크롤링

## 검색명이 담겨있는 데이터 불러오기

In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('data/이슈크롤링_소분류별_검색명(전체).xlsx')
data

,소분류,검색명
0,LED/전구/조명,LED 전등
1,건강즙,건강즙
2,TV거실장,TV거실장
3,니트/가디건,니트
4,가스/전기레인지,전기레인지
...,...,...
188,헤어케어,헤어케어
189,혈행건강,혈행건강
190,프라이팬,프라이팬
191,홍삼,홍삼


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   소분류     193 non-null    object
 1   검색명     193 non-null    object
dtypes: object(2)
memory usage: 3.1+ KB


## Train data에 반영할 데이터 수집 (20181217 ~ 20181231 + 20190101 ~ 2019122)
- 2주전 데이터 반영할 예정이라 20191223 ~ 20191231 제외됨

### 크롤링 날짜 list 생성

In [4]:
day_list = pd.date_range(start='12/17/18', end = '12/22/19')   # 월/일/년
day_list

DatetimeIndex(['2018-12-17', '2018-12-18', '2018-12-19', '2018-12-20',
               '2018-12-21', '2018-12-22', '2018-12-23', '2018-12-24',
               '2018-12-25', '2018-12-26',
               ...
               '2019-12-13', '2019-12-14', '2019-12-15', '2019-12-16',
               '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20',
               '2019-12-21', '2019-12-22'],
              dtype='datetime64[ns]', length=371, freq='D')

In [5]:
# 위에 그대로 돌리면 '2019-01-01 00:00:00' 이런 형태로 반환됨
for day in day_list:
    print(day)

2018-12-17 00:00:00
2018-12-18 00:00:00
2018-12-19 00:00:00
2018-12-20 00:00:00
2018-12-21 00:00:00
2018-12-22 00:00:00
2018-12-23 00:00:00
2018-12-24 00:00:00
2018-12-25 00:00:00
2018-12-26 00:00:00
2018-12-27 00:00:00
2018-12-28 00:00:00
2018-12-29 00:00:00
2018-12-30 00:00:00
2018-12-31 00:00:00
2019-01-01 00:00:00
2019-01-02 00:00:00
2019-01-03 00:00:00
2019-01-04 00:00:00
2019-01-05 00:00:00
2019-01-06 00:00:00
2019-01-07 00:00:00
2019-01-08 00:00:00
2019-01-09 00:00:00
2019-01-10 00:00:00
2019-01-11 00:00:00
2019-01-12 00:00:00
2019-01-13 00:00:00
2019-01-14 00:00:00
2019-01-15 00:00:00
2019-01-16 00:00:00
2019-01-17 00:00:00
2019-01-18 00:00:00
2019-01-19 00:00:00
2019-01-20 00:00:00
2019-01-21 00:00:00
2019-01-22 00:00:00
2019-01-23 00:00:00
2019-01-24 00:00:00
2019-01-25 00:00:00
2019-01-26 00:00:00
2019-01-27 00:00:00
2019-01-28 00:00:00
2019-01-29 00:00:00
2019-01-30 00:00:00
2019-01-31 00:00:00
2019-02-01 00:00:00
2019-02-02 00:00:00
2019-02-03 00:00:00
2019-02-04 00:00:00


In [6]:
type(day_list)

pandas.core.indexes.datetimes.DatetimeIndex

In [7]:
# 날짜만 뽑아서, string type으로 변경
day_list = list(map(str, day_list.date))
day_list

['2018-12-17',
 '2018-12-18',
 '2018-12-19',
 '2018-12-20',
 '2018-12-21',
 '2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28',
 '2018-12-29',
 '2018-12-30',
 '2018-12-31',
 '2019-01-01',
 '2019-01-02',
 '2019-01-03',
 '2019-01-04',
 '2019-01-05',
 '2019-01-06',
 '2019-01-07',
 '2019-01-08',
 '2019-01-09',
 '2019-01-10',
 '2019-01-11',
 '2019-01-12',
 '2019-01-13',
 '2019-01-14',
 '2019-01-15',
 '2019-01-16',
 '2019-01-17',
 '2019-01-18',
 '2019-01-19',
 '2019-01-20',
 '2019-01-21',
 '2019-01-22',
 '2019-01-23',
 '2019-01-24',
 '2019-01-25',
 '2019-01-26',
 '2019-01-27',
 '2019-01-28',
 '2019-01-29',
 '2019-01-30',
 '2019-01-31',
 '2019-02-01',
 '2019-02-02',
 '2019-02-03',
 '2019-02-04',
 '2019-02-05',
 '2019-02-06',
 '2019-02-07',
 '2019-02-08',
 '2019-02-09',
 '2019-02-10',
 '2019-02-11',
 '2019-02-12',
 '2019-02-13',
 '2019-02-14',
 '2019-02-15',
 '2019-02-16',
 '2019-02-17',
 '2019-02-18',
 '2019-02-19',
 '2019-02-20',
 '2019-02-

In [8]:
# day_type1: '2019.01.01' 형태
day_type1_list = list(map(lambda x : x.replace('-','.'), day_list))
day_type1_list

['2018.12.17',
 '2018.12.18',
 '2018.12.19',
 '2018.12.20',
 '2018.12.21',
 '2018.12.22',
 '2018.12.23',
 '2018.12.24',
 '2018.12.25',
 '2018.12.26',
 '2018.12.27',
 '2018.12.28',
 '2018.12.29',
 '2018.12.30',
 '2018.12.31',
 '2019.01.01',
 '2019.01.02',
 '2019.01.03',
 '2019.01.04',
 '2019.01.05',
 '2019.01.06',
 '2019.01.07',
 '2019.01.08',
 '2019.01.09',
 '2019.01.10',
 '2019.01.11',
 '2019.01.12',
 '2019.01.13',
 '2019.01.14',
 '2019.01.15',
 '2019.01.16',
 '2019.01.17',
 '2019.01.18',
 '2019.01.19',
 '2019.01.20',
 '2019.01.21',
 '2019.01.22',
 '2019.01.23',
 '2019.01.24',
 '2019.01.25',
 '2019.01.26',
 '2019.01.27',
 '2019.01.28',
 '2019.01.29',
 '2019.01.30',
 '2019.01.31',
 '2019.02.01',
 '2019.02.02',
 '2019.02.03',
 '2019.02.04',
 '2019.02.05',
 '2019.02.06',
 '2019.02.07',
 '2019.02.08',
 '2019.02.09',
 '2019.02.10',
 '2019.02.11',
 '2019.02.12',
 '2019.02.13',
 '2019.02.14',
 '2019.02.15',
 '2019.02.16',
 '2019.02.17',
 '2019.02.18',
 '2019.02.19',
 '2019.02.20',
 '2019.02.

In [9]:
len(day_type1_list)

371

In [10]:
# day_type2: '20190101'형태
day_type2_list = list(map(lambda x : x.replace('-',''), day_list))
day_type2_list

['20181217',
 '20181218',
 '20181219',
 '20181220',
 '20181221',
 '20181222',
 '20181223',
 '20181224',
 '20181225',
 '20181226',
 '20181227',
 '20181228',
 '20181229',
 '20181230',
 '20181231',
 '20190101',
 '20190102',
 '20190103',
 '20190104',
 '20190105',
 '20190106',
 '20190107',
 '20190108',
 '20190109',
 '20190110',
 '20190111',
 '20190112',
 '20190113',
 '20190114',
 '20190115',
 '20190116',
 '20190117',
 '20190118',
 '20190119',
 '20190120',
 '20190121',
 '20190122',
 '20190123',
 '20190124',
 '20190125',
 '20190126',
 '20190127',
 '20190128',
 '20190129',
 '20190130',
 '20190131',
 '20190201',
 '20190202',
 '20190203',
 '20190204',
 '20190205',
 '20190206',
 '20190207',
 '20190208',
 '20190209',
 '20190210',
 '20190211',
 '20190212',
 '20190213',
 '20190214',
 '20190215',
 '20190216',
 '20190217',
 '20190218',
 '20190219',
 '20190220',
 '20190221',
 '20190222',
 '20190223',
 '20190224',
 '20190225',
 '20190226',
 '20190227',
 '20190228',
 '20190301',
 '20190302',
 '20190303',

In [11]:
len(day_type2_list)

371

### 크롤링 함수 생성

- 건수: div.title_desc.all_my : text

In [12]:
test_url = 'https://search.naver.com/search.naver?where=news&query=tv&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds=2019.01.01&de=2019.01.20&docid=&nso=so%3Ar%2Cp%3Afrom20190101to20190120%2Ca%3Aall&mynews=0&refresh_start=0&related=0'

In [13]:
from bs4 import BeautifulSoup
import requests

def get_naver_news_cnt(url):
    error_cnt = 0   # 요청 error count
    # 요청
    res = requests.get(url)
    if res.status_code == 200:
        try:
            soup = BeautifulSoup(res.text)
            # 건수 가져오기
            cnt = soup.select_one('div.title_desc.all_my').text.strip()
            # '1-10 / 43,302건'   ->   '/'로 분할, 공백제거, '건'제거 
            cnt = cnt.split('/')[1].strip().replace('건','')
            # 건수에서 쉼표 제거
            cnt = ''.join(cnt.split(','))
            return cnt
        except:
            # 검색 결과 건수 없을 때
            return 0
    else:
        error_cnt += 1
#         raise Exception("요청 실패: {}".format(res.status_code))

In [14]:
get_naver_news_cnt(test_url)

'43305'

In [15]:
# 검색결과 없을때
test_url2 = 'https://search.naver.com/search.naver?where=news&query=퉹&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds=2019.01.21&de=2019.01.31&docid=&nso=so%3Ar%2Cp%3Afrom20190121to20190131%2Ca%3Aall&mynews=0&refresh_start=0&related=0'
get_naver_news_cnt(test_url2)

0

### 검색키워드별 일별 건수 수집 진행
- 결과 담을 list -> 나중에 dataframe으로 변환
    - result.append([item, keyword, day, cnt]) ['소분류','검색명','날짜','기사건수']

In [16]:
base_url = 'https://search.naver.com/search.naver?where=news&query={}&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={}&de={}&docid=&nso=so%3Ar%2Cp%3Afrom{}to{}%2Ca%3Aall&mynews=0&refresh_start=0&related=0'

In [19]:
# 10시간 소요됨

result = []
for i in data.index: # 0 ~ 193
    for day_type1, day_type2 in zip(day_type1_list, day_type2_list):
        # 검색명 & 날짜에 맞는 url 생성
        url = base_url.format(data.loc[i,'검색명'], day_type1, day_type1, day_type2, day_type2)
        # 수집 & 기사건수 반환
        cnt = get_naver_news_cnt(url)
        day = day_type1.replace('.','-')
        # list에 append
        result.append([data.loc[i,'소분류'], data.loc[i,'검색명'], day, cnt])

### 수집결과 정리

In [20]:
result

[['LED/전구/조명', 'LED 전등', '2018-12-17', '9'],
 ['LED/전구/조명', 'LED 전등', '2018-12-18', '5'],
 ['LED/전구/조명', 'LED 전등', '2018-12-19', '10'],
 ['LED/전구/조명', 'LED 전등', '2018-12-20', '10'],
 ['LED/전구/조명', 'LED 전등', '2018-12-21', 0],
 ['LED/전구/조명', 'LED 전등', '2018-12-22', '3'],
 ['LED/전구/조명', 'LED 전등', '2018-12-23', 0],
 ['LED/전구/조명', 'LED 전등', '2018-12-24', '10'],
 ['LED/전구/조명', 'LED 전등', '2018-12-25', '4'],
 ['LED/전구/조명', 'LED 전등', '2018-12-26', '1'],
 ['LED/전구/조명', 'LED 전등', '2018-12-27', '5'],
 ['LED/전구/조명', 'LED 전등', '2018-12-28', '5'],
 ['LED/전구/조명', 'LED 전등', '2018-12-29', 0],
 ['LED/전구/조명', 'LED 전등', '2018-12-30', '6'],
 ['LED/전구/조명', 'LED 전등', '2018-12-31', '8'],
 ['LED/전구/조명', 'LED 전등', '2019-01-01', '1'],
 ['LED/전구/조명', 'LED 전등', '2019-01-02', '10'],
 ['LED/전구/조명', 'LED 전등', '2019-01-03', '10'],
 ['LED/전구/조명', 'LED 전등', '2019-01-04', '2'],
 ['LED/전구/조명', 'LED 전등', '2019-01-05', '2'],
 ['LED/전구/조명', 'LED 전등', '2019-01-06', '2'],
 ['LED/전구/조명', 'LED 전등', '2019-01-07', '10'],
 ['LED/전구/

In [22]:
result = pd.DataFrame(result, columns = ['소분류','검색명','날짜','기사건수'])
result

,소분류,검색명,날짜,기사건수
0,LED/전구/조명,LED 전등,2018-12-17,9
1,LED/전구/조명,LED 전등,2018-12-18,5
2,LED/전구/조명,LED 전등,2018-12-19,10
3,LED/전구/조명,LED 전등,2018-12-20,10
4,LED/전구/조명,LED 전등,2018-12-21,0
...,...,...,...,...
51193,화장대/협탁,협탁,2019-12-18,4
51194,화장대/협탁,협탁,2019-12-19,0
51195,화장대/협탁,협탁,2019-12-20,3
51196,화장대/협탁,협탁,2019-12-21,3


In [23]:
result['소분류'].value_counts()

커피/생수/음료    1113
욕실/목욕용품      742
커튼/블라인드      742
구강용품/칫솔      742
면도용품/면도기     742
            ... 
패션 ACC       371
의자           371
여성가방/핸드백     371
카페트/러그       371
자켓/점퍼/코트     371
Name: 소분류, Length: 114, dtype: int64

In [24]:
# 기사건수 type변경
result['기사건수'] = result['기사건수'].astype(int)

### groupby

In [15]:
# groupby로 최종 데이터 형태 만들기 (소분류 검색명 여러개인 것들을 하나로 합치는 작업)
final = result.groupby(['소분류','날짜'])[['기사건수']].sum().reset_index()
final

,소분류,날짜,기사건수
0,LED/전구/조명,2018-12-17,9
1,LED/전구/조명,2018-12-18,5
2,LED/전구/조명,2018-12-19,10
3,LED/전구/조명,2018-12-20,10
4,LED/전구/조명,2018-12-21,0
...,...,...,...
54532,화장대/협탁,2019-12-18,8
54533,화장대/협탁,2019-12-19,10
54534,화장대/협탁,2019-12-20,5
54535,화장대/협탁,2019-12-21,8


In [17]:
final['소분류'].value_counts().sort_values() # 371개씩 있어야함

물매트/냉감패드    371
원예/조경용품     371
팬츠/데님       371
카페트/러그      371
서랍장         371
           ... 
셔츠/남방       371
웨이트기구       371
아동속옷        371
다리미         371
김치/반찬       371
Name: 소분류, Length: 147, dtype: int64

In [18]:
# 저장
final.to_csv('data/소분류별_뉴스건수크롤링(20181217~20191222).csv', index=False)

## Test 데이터에 반영할 데이터 수집(20200518 ~20200630)

### 크롤링 날짜 list 생성

In [51]:
data = pd.read_excel('data/이슈크롤링_소분류별_검색명(전체).xlsx')
data

,소분류,검색명
0,LED/전구/조명,LED 전등
1,건강즙,건강즙
2,TV거실장,TV거실장
3,니트/가디건,니트
4,가스/전기레인지,전기레인지
...,...,...
188,헤어케어,헤어케어
189,혈행건강,혈행건강
190,프라이팬,프라이팬
191,홍삼,홍삼


In [52]:
day_list_2020 = pd.date_range(start='05/18/20', end = '06/30/20')   # 월/일/년
day_list_2020

DatetimeIndex(['2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21',
               '2020-05-22', '2020-05-23', '2020-05-24', '2020-05-25',
               '2020-05-26', '2020-05-27', '2020-05-28', '2020-05-29',
               '2020-05-30', '2020-05-31', '2020-06-01', '2020-06-02',
               '2020-06-03', '2020-06-04', '2020-06-05', '2020-06-06',
               '2020-06-07', '2020-06-08', '2020-06-09', '2020-06-10',
               '2020-06-11', '2020-06-12', '2020-06-13', '2020-06-14',
               '2020-06-15', '2020-06-16', '2020-06-17', '2020-06-18',
               '2020-06-19', '2020-06-20', '2020-06-21', '2020-06-22',
               '2020-06-23', '2020-06-24', '2020-06-25', '2020-06-26',
               '2020-06-27', '2020-06-28', '2020-06-29', '2020-06-30'],
              dtype='datetime64[ns]', freq='D')

In [53]:
# 위에 그대로 돌리면 '2019-01-01 00:00:00' 이런 형태로 반환됨
for day in day_list_2020:
    print(day)

2020-05-18 00:00:00
2020-05-19 00:00:00
2020-05-20 00:00:00
2020-05-21 00:00:00
2020-05-22 00:00:00
2020-05-23 00:00:00
2020-05-24 00:00:00
2020-05-25 00:00:00
2020-05-26 00:00:00
2020-05-27 00:00:00
2020-05-28 00:00:00
2020-05-29 00:00:00
2020-05-30 00:00:00
2020-05-31 00:00:00
2020-06-01 00:00:00
2020-06-02 00:00:00
2020-06-03 00:00:00
2020-06-04 00:00:00
2020-06-05 00:00:00
2020-06-06 00:00:00
2020-06-07 00:00:00
2020-06-08 00:00:00
2020-06-09 00:00:00
2020-06-10 00:00:00
2020-06-11 00:00:00
2020-06-12 00:00:00
2020-06-13 00:00:00
2020-06-14 00:00:00
2020-06-15 00:00:00
2020-06-16 00:00:00
2020-06-17 00:00:00
2020-06-18 00:00:00
2020-06-19 00:00:00
2020-06-20 00:00:00
2020-06-21 00:00:00
2020-06-22 00:00:00
2020-06-23 00:00:00
2020-06-24 00:00:00
2020-06-25 00:00:00
2020-06-26 00:00:00
2020-06-27 00:00:00
2020-06-28 00:00:00
2020-06-29 00:00:00
2020-06-30 00:00:00


In [54]:
type(day_list_2020)

pandas.core.indexes.datetimes.DatetimeIndex

In [55]:
# 날짜만 뽑아서, string type으로 변경
day_list_2020 = list(map(str, day_list_2020.date))
day_list_2020

['2020-05-18',
 '2020-05-19',
 '2020-05-20',
 '2020-05-21',
 '2020-05-22',
 '2020-05-23',
 '2020-05-24',
 '2020-05-25',
 '2020-05-26',
 '2020-05-27',
 '2020-05-28',
 '2020-05-29',
 '2020-05-30',
 '2020-05-31',
 '2020-06-01',
 '2020-06-02',
 '2020-06-03',
 '2020-06-04',
 '2020-06-05',
 '2020-06-06',
 '2020-06-07',
 '2020-06-08',
 '2020-06-09',
 '2020-06-10',
 '2020-06-11',
 '2020-06-12',
 '2020-06-13',
 '2020-06-14',
 '2020-06-15',
 '2020-06-16',
 '2020-06-17',
 '2020-06-18',
 '2020-06-19',
 '2020-06-20',
 '2020-06-21',
 '2020-06-22',
 '2020-06-23',
 '2020-06-24',
 '2020-06-25',
 '2020-06-26',
 '2020-06-27',
 '2020-06-28',
 '2020-06-29',
 '2020-06-30']

In [56]:
# day_type1: '2019.01.01' 형태
day2020_type1_list = list(map(lambda x : x.replace('-','.'), day_list_2020))
day2020_type1_list

['2020.05.18',
 '2020.05.19',
 '2020.05.20',
 '2020.05.21',
 '2020.05.22',
 '2020.05.23',
 '2020.05.24',
 '2020.05.25',
 '2020.05.26',
 '2020.05.27',
 '2020.05.28',
 '2020.05.29',
 '2020.05.30',
 '2020.05.31',
 '2020.06.01',
 '2020.06.02',
 '2020.06.03',
 '2020.06.04',
 '2020.06.05',
 '2020.06.06',
 '2020.06.07',
 '2020.06.08',
 '2020.06.09',
 '2020.06.10',
 '2020.06.11',
 '2020.06.12',
 '2020.06.13',
 '2020.06.14',
 '2020.06.15',
 '2020.06.16',
 '2020.06.17',
 '2020.06.18',
 '2020.06.19',
 '2020.06.20',
 '2020.06.21',
 '2020.06.22',
 '2020.06.23',
 '2020.06.24',
 '2020.06.25',
 '2020.06.26',
 '2020.06.27',
 '2020.06.28',
 '2020.06.29',
 '2020.06.30']

In [57]:
len(day2020_type1_list)

44

In [58]:
# day_type2: '20190101'형태
day2020_type2_list = list(map(lambda x : x.replace('-',''), day_list_2020))
day2020_type2_list

['20200518',
 '20200519',
 '20200520',
 '20200521',
 '20200522',
 '20200523',
 '20200524',
 '20200525',
 '20200526',
 '20200527',
 '20200528',
 '20200529',
 '20200530',
 '20200531',
 '20200601',
 '20200602',
 '20200603',
 '20200604',
 '20200605',
 '20200606',
 '20200607',
 '20200608',
 '20200609',
 '20200610',
 '20200611',
 '20200612',
 '20200613',
 '20200614',
 '20200615',
 '20200616',
 '20200617',
 '20200618',
 '20200619',
 '20200620',
 '20200621',
 '20200622',
 '20200623',
 '20200624',
 '20200625',
 '20200626',
 '20200627',
 '20200628',
 '20200629',
 '20200630']

In [59]:
len(day2020_type2_list)

44

### 수집 진행

In [60]:
base_url = 'https://search.naver.com/search.naver?where=news&query={}&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={}&de={}&docid=&nso=so%3Ar%2Cp%3Afrom{}to{}%2Ca%3Aall&mynews=0&refresh_start=0&related=0'

In [61]:
# 1시간 30분정도 소요
result = []
for i in data.index:   # 0 ~ 193
    for day_type1, day_type2 in zip(day2020_type1_list, day2020_type2_list):
        # 검색명 & 날짜에 맞는 url 생성
        url = base_url.format(data.loc[i,'검색명'], day_type1, day_type1, day_type2, day_type2)
        # 수집 & 기사건수 반환
        cnt = get_naver_news_cnt(url)
        day = day_type1.replace('.','-')
        # list에 append
        result.append([data.loc[i,'소분류'], data.loc[i,'검색명'], day, cnt])

### 수집결과 정리

In [62]:
result

[['LED/전구/조명', 'LED 전등', '2020-05-18', '4'],
 ['LED/전구/조명', 'LED 전등', '2020-05-19', '3'],
 ['LED/전구/조명', 'LED 전등', '2020-05-20', '8'],
 ['LED/전구/조명', 'LED 전등', '2020-05-21', '1'],
 ['LED/전구/조명', 'LED 전등', '2020-05-22', '1'],
 ['LED/전구/조명', 'LED 전등', '2020-05-23', 0],
 ['LED/전구/조명', 'LED 전등', '2020-05-24', '2'],
 ['LED/전구/조명', 'LED 전등', '2020-05-25', '10'],
 ['LED/전구/조명', 'LED 전등', '2020-05-26', '10'],
 ['LED/전구/조명', 'LED 전등', '2020-05-27', '10'],
 ['LED/전구/조명', 'LED 전등', '2020-05-28', '7'],
 ['LED/전구/조명', 'LED 전등', '2020-05-29', '10'],
 ['LED/전구/조명', 'LED 전등', '2020-05-30', 0],
 ['LED/전구/조명', 'LED 전등', '2020-05-31', '7'],
 ['LED/전구/조명', 'LED 전등', '2020-06-01', '10'],
 ['LED/전구/조명', 'LED 전등', '2020-06-02', '5'],
 ['LED/전구/조명', 'LED 전등', '2020-06-03', '6'],
 ['LED/전구/조명', 'LED 전등', '2020-06-04', '10'],
 ['LED/전구/조명', 'LED 전등', '2020-06-05', '5'],
 ['LED/전구/조명', 'LED 전등', '2020-06-06', 0],
 ['LED/전구/조명', 'LED 전등', '2020-06-07', '10'],
 ['LED/전구/조명', 'LED 전등', '2020-06-08', '7'],
 ['LED/전구

In [64]:
result = pd.DataFrame(result, columns = ['소분류','검색명','날짜','기사건수'])
result

,소분류,검색명,날짜,기사건수
0,LED/전구/조명,LED 전등,2020-05-18,4
1,LED/전구/조명,LED 전등,2020-05-19,3
2,LED/전구/조명,LED 전등,2020-05-20,8
3,LED/전구/조명,LED 전등,2020-05-21,1
4,LED/전구/조명,LED 전등,2020-05-22,1
...,...,...,...,...
8487,화장대/협탁,협탁,2020-06-26,1
8488,화장대/협탁,협탁,2020-06-27,0
8489,화장대/협탁,협탁,2020-06-28,1
8490,화장대/협탁,협탁,2020-06-29,0


In [65]:
# 기사건수 type변경
result['기사건수'] = result['기사건수'].astype(int)

In [66]:
result['소분류'].value_counts()

커피/생수/음료        132
트레킹화/러닝화/워킹화    132
가죽/모피/무스탕       132
세탁/건조/스타일러      132
자켓/점퍼/코트        132
               ... 
TV               44
기초케어             44
웨이트기구            44
LED/전구/조명        44
다리미              44
Name: 소분류, Length: 147, dtype: int64

### groupby

In [68]:
# groupby로 최종 데이터 형태 만들기 (소분류 검색명 여러개인거 하나로 합치는 작업)
final = result.groupby(['소분류','날짜'])[['기사건수']].sum().reset_index()
final

,소분류,날짜,기사건수
0,LED/전구/조명,2020-05-18,4
1,LED/전구/조명,2020-05-19,3
2,LED/전구/조명,2020-05-20,8
3,LED/전구/조명,2020-05-21,1
4,LED/전구/조명,2020-05-22,1
...,...,...,...
6463,화장대/협탁,2020-06-26,11
6464,화장대/협탁,2020-06-27,4
6465,화장대/협탁,2020-06-28,8
6466,화장대/협탁,2020-06-29,10


In [69]:
# 저장
final.to_csv('data/소분류별_뉴스건수크롤링(20200518~20200630).csv', index=False)

# Train data에 '뉴스건수' 컬럼추가

## 소분류별 20191223 ~ 20191231 행 추가
- 2주전의 데이터 적용으로 크롤링에서는 필요 없었으나, train data에서 소분류+날짜 기준으로 매칭(join)할 때 필요함

In [110]:
import pandas as pd
# 소분류 컬럼만 사용할 예정
data = pd.read_excel('data/이슈크롤링_소분류별_검색명(전체).xlsx')
data

,소분류,검색명
0,LED/전구/조명,LED 전등
1,건강즙,건강즙
2,TV거실장,TV거실장
3,니트/가디건,니트
4,가스/전기레인지,전기레인지
...,...,...
188,헤어케어,헤어케어
189,혈행건강,혈행건강
190,프라이팬,프라이팬
191,홍삼,홍삼


In [111]:
day_list = pd.date_range(start='12/23/19', end = '12/31/19')   # 월/일/년
day_list

DatetimeIndex(['2019-12-23', '2019-12-24', '2019-12-25', '2019-12-26',
               '2019-12-27', '2019-12-28', '2019-12-29', '2019-12-30',
               '2019-12-31'],
              dtype='datetime64[ns]', freq='D')

In [112]:
type(day_list), len(day_list)

(pandas.core.indexes.datetimes.DatetimeIndex, 9)

In [113]:
# 날짜만 뽑아서, string type으로 변경
day_list = list(map(str, day_list.date))
day_list

['2019-12-23',
 '2019-12-24',
 '2019-12-25',
 '2019-12-26',
 '2019-12-27',
 '2019-12-28',
 '2019-12-29',
 '2019-12-30',
 '2019-12-31']

In [114]:
len(day_list)

9

In [115]:
add_result = []
for i in data.index:
    for day in day_list:
        mini = data.loc[i,'소분류']
        add_result.append([mini, day])

In [116]:
add_df = pd.DataFrame(add_result, columns=['소분류','날짜'])
add_df

,소분류,날짜
0,LED/전구/조명,2019-12-23
1,LED/전구/조명,2019-12-24
2,LED/전구/조명,2019-12-25
3,LED/전구/조명,2019-12-26
4,LED/전구/조명,2019-12-27
...,...,...
1732,화장대/협탁,2019-12-27
1733,화장대/협탁,2019-12-28
1734,화장대/협탁,2019-12-29
1735,화장대/협탁,2019-12-30


In [117]:
193*9

1737

In [118]:
# 기사건수 열 추가 (nan값으로)
add_df['기사건수'] = np.nan
add_df

NameError: name 'np' is not defined

## 크롤링 데이터와 concat

In [119]:
final = pd.read_csv('data/소분류별_뉴스건수크롤링(20181217~20191222).csv')
final

,소분류,날짜,기사건수
0,LED/전구/조명,2018-12-17,9
1,LED/전구/조명,2018-12-18,5
2,LED/전구/조명,2018-12-19,10
3,LED/전구/조명,2018-12-20,10
4,LED/전구/조명,2018-12-21,0
...,...,...,...
54532,화장대/협탁,2019-12-18,8
54533,화장대/협탁,2019-12-19,10
54534,화장대/협탁,2019-12-20,5
54535,화장대/협탁,2019-12-21,8


In [120]:
54537 + 1737 

56274

In [121]:
final = pd.concat([add_df, final]).sort_values(by=['소분류','날짜']).reset_index()
final.drop('index',axis=1, inplace=True)
final

,소분류,날짜,기사건수
0,LED/전구/조명,2018-12-17,9.0
1,LED/전구/조명,2018-12-18,5.0
2,LED/전구/조명,2018-12-19,10.0
3,LED/전구/조명,2018-12-20,10.0
4,LED/전구/조명,2018-12-21,0.0
...,...,...,...
56269,화장대/협탁,2019-12-29,NaN
56270,화장대/협탁,2019-12-30,NaN
56271,화장대/협탁,2019-12-30,NaN
56272,화장대/협탁,2019-12-31,NaN


## 데이터 적용 형태로 변경 (일별 -> 주 단위)

In [122]:
import datetime

In [123]:
# 날짜 type object -> datetime으로 변경
final['날짜'] = pd.to_datetime(final['날짜'])

In [124]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56274 entries, 0 to 56273
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   소분류     56274 non-null  object        
 1   날짜      56274 non-null  datetime64[ns]
 2   기사건수    54537 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.3+ MB


- 월요일이 한주의 시작 기준

In [125]:
final['실제주'] = final['날짜'].map(lambda x: x.isocalendar()[1])

In [126]:
final

,소분류,날짜,기사건수,실제주
0,LED/전구/조명,2018-12-17,9.0,51
1,LED/전구/조명,2018-12-18,5.0,51
2,LED/전구/조명,2018-12-19,10.0,51
3,LED/전구/조명,2018-12-20,10.0,51
4,LED/전구/조명,2018-12-21,0.0,51
...,...,...,...,...
56269,화장대/협탁,2019-12-29,NaN,52
56270,화장대/협탁,2019-12-30,NaN,1
56271,화장대/협탁,2019-12-30,NaN,1
56272,화장대/협탁,2019-12-31,NaN,1


In [127]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56274 entries, 0 to 56273
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   소분류     56274 non-null  object        
 1   날짜      56274 non-null  datetime64[ns]
 2   기사건수    54537 non-null  float64       
 3   실제주     56274 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [128]:
import numpy as np
final['2주전'] = np.nan
final

,소분류,날짜,기사건수,실제주,2주전
0,LED/전구/조명,2018-12-17,9.0,51,NaN
1,LED/전구/조명,2018-12-18,5.0,51,NaN
2,LED/전구/조명,2018-12-19,10.0,51,NaN
3,LED/전구/조명,2018-12-20,10.0,51,NaN
4,LED/전구/조명,2018-12-21,0.0,51,NaN
...,...,...,...,...,...
56269,화장대/협탁,2019-12-29,NaN,52,NaN
56270,화장대/협탁,2019-12-30,NaN,1,NaN
56271,화장대/협탁,2019-12-30,NaN,1,NaN
56272,화장대/협탁,2019-12-31,NaN,1,NaN


In [129]:
# 20190101 부터 계산 : 실제주 - 2 if 실제주 == 2 return 52 elif 실제주 == 1 return 51
def week_2(x):
    if x == 1:
        return 51
    elif x == 2:
        return 52
    else:
        return x - 2

In [130]:
final['2주전'] = final['실제주'].map(lambda x: week_2(x))

In [131]:
final

,소분류,날짜,기사건수,실제주,2주전
0,LED/전구/조명,2018-12-17,9.0,51,49
1,LED/전구/조명,2018-12-18,5.0,51,49
2,LED/전구/조명,2018-12-19,10.0,51,49
3,LED/전구/조명,2018-12-20,10.0,51,49
4,LED/전구/조명,2018-12-21,0.0,51,49
...,...,...,...,...,...
56269,화장대/협탁,2019-12-29,NaN,52,50
56270,화장대/협탁,2019-12-30,NaN,1,51
56271,화장대/협탁,2019-12-30,NaN,1,51
56272,화장대/협탁,2019-12-31,NaN,1,51


## '2주전_기사건수' 컬럼 추가
- 주 단위로 기사 건수 합치기
- tmp에서는 실제주를, final에서는 2주전주를 join_key에 넣어서 2주전 기사건수 컬럼 추가할 예정
- 2018년도와 2019년도 중 겹치는 주가 있어(ex. 51주는 2018, 2019년도 둘다 존재), join_key에 년도도 추가

In [132]:
tmp = final.copy()
tmp

,소분류,날짜,기사건수,실제주,2주전
0,LED/전구/조명,2018-12-17,9.0,51,49
1,LED/전구/조명,2018-12-18,5.0,51,49
2,LED/전구/조명,2018-12-19,10.0,51,49
3,LED/전구/조명,2018-12-20,10.0,51,49
4,LED/전구/조명,2018-12-21,0.0,51,49
...,...,...,...,...,...
56269,화장대/협탁,2019-12-29,NaN,52,50
56270,화장대/협탁,2019-12-30,NaN,1,51
56271,화장대/협탁,2019-12-30,NaN,1,51
56272,화장대/협탁,2019-12-31,NaN,1,51


In [133]:
tmp.drop('2주전', axis=1, inplace=True)
tmp

,소분류,날짜,기사건수,실제주
0,LED/전구/조명,2018-12-17,9.0,51
1,LED/전구/조명,2018-12-18,5.0,51
2,LED/전구/조명,2018-12-19,10.0,51
3,LED/전구/조명,2018-12-20,10.0,51
4,LED/전구/조명,2018-12-21,0.0,51
...,...,...,...,...
56269,화장대/협탁,2019-12-29,NaN,52
56270,화장대/협탁,2019-12-30,NaN,1
56271,화장대/협탁,2019-12-30,NaN,1
56272,화장대/협탁,2019-12-31,NaN,1


In [134]:
# final에는 2019년 데이터만 남기기
final = final[final['날짜'] >= '20190101'].copy()
final

,소분류,날짜,기사건수,실제주,2주전
15,LED/전구/조명,2019-01-01,1.0,1,51
16,LED/전구/조명,2019-01-02,10.0,1,51
17,LED/전구/조명,2019-01-03,10.0,1,51
18,LED/전구/조명,2019-01-04,2.0,1,51
19,LED/전구/조명,2019-01-05,2.0,1,51
...,...,...,...,...,...
56269,화장대/협탁,2019-12-29,NaN,52,50
56270,화장대/협탁,2019-12-30,NaN,1,51
56271,화장대/협탁,2019-12-30,NaN,1,51
56272,화장대/협탁,2019-12-31,NaN,1,51


## join key 생성

In [135]:
tmp['실제주'] = tmp['실제주'].astype(str)
final['2주전'] = final['2주전'].astype(str)

In [136]:
tmp['년도'] = tmp['날짜'].dt.year
final['년도'] = final['날짜'].dt.year

In [137]:
tmp['년도'] = tmp['년도'].astype(str)
final['년도'] = final['년도'].astype(str)

In [138]:
tmp['join_key'] = tmp['소분류']+tmp['년도']+ tmp['실제주']
tmp

,소분류,날짜,기사건수,실제주,년도,join_key
0,LED/전구/조명,2018-12-17,9.0,51,2018,LED/전구/조명201851
1,LED/전구/조명,2018-12-18,5.0,51,2018,LED/전구/조명201851
2,LED/전구/조명,2018-12-19,10.0,51,2018,LED/전구/조명201851
3,LED/전구/조명,2018-12-20,10.0,51,2018,LED/전구/조명201851
4,LED/전구/조명,2018-12-21,0.0,51,2018,LED/전구/조명201851
...,...,...,...,...,...,...
56269,화장대/협탁,2019-12-29,NaN,52,2019,화장대/협탁201952
56270,화장대/협탁,2019-12-30,NaN,1,2019,화장대/협탁20191
56271,화장대/협탁,2019-12-30,NaN,1,2019,화장대/협탁20191
56272,화장대/협탁,2019-12-31,NaN,1,2019,화장대/협탁20191


In [139]:
# join에 필요한 컬럼만 남기기
tmp = tmp[['join_key','기사건수']].copy()
tmp

,join_key,기사건수
0,LED/전구/조명201851,9.0
1,LED/전구/조명201851,5.0
2,LED/전구/조명201851,10.0
3,LED/전구/조명201851,10.0
4,LED/전구/조명201851,0.0
...,...,...
56269,화장대/협탁201952,NaN
56270,화장대/협탁20191,NaN
56271,화장대/협탁20191,NaN
56272,화장대/협탁20191,NaN


In [140]:
tmp.columns = ['join_key','2주전_기사건수']
tmp

,join_key,2주전_기사건수
0,LED/전구/조명201851,9.0
1,LED/전구/조명201851,5.0
2,LED/전구/조명201851,10.0
3,LED/전구/조명201851,10.0
4,LED/전구/조명201851,0.0
...,...,...
56269,화장대/협탁201952,NaN
56270,화장대/협탁20191,NaN
56271,화장대/협탁20191,NaN
56272,화장대/협탁20191,NaN


In [141]:
# groupby해서 주 단위로 합치기
tmp = tmp.groupby(['join_key'])[['2주전_기사건수']].sum()
tmp

,2주전_기사건수
join_key,
LED/전구/조명20181,8.0
LED/전구/조명201851,37.0
LED/전구/조명201852,31.0
LED/전구/조명20191,27.0
LED/전구/조명201910,16.0
...,...
화장대/협탁201952,0.0
화장대/협탁20196,47.0
화장대/협탁20197,45.0


In [142]:
# 2019년도 1월의 2주전 값이 51, 52인 것은 '년도' 컬럼이 2019가 아닌, 2018로 변경해야 정상 join 가능
final.loc[(final['날짜'] <= '20190131') & (final['2주전'] == '51'),'년도'].unique()

array(['2019'], dtype=object)

In [143]:
final.loc[(final['날짜'] <= '20190131') & (final['2주전'] == '51'),'년도'] = '2018'
final.loc[(final['날짜'] <= '20190131') & (final['2주전'] == '52'),'년도'] = '2018'

In [144]:
final.loc[(final['날짜'] <= '20190131') & (final['2주전'] == '51'),'년도'].unique()

array(['2018'], dtype=object)

In [145]:
final.loc[(final['날짜'] <= '20190131') & (final['2주전'] == '52'),'년도'].unique()

array(['2018'], dtype=object)

In [146]:
final['join_key'] = final['소분류']+final['년도']+ final['2주전']
final

,소분류,날짜,기사건수,실제주,2주전,년도,join_key
15,LED/전구/조명,2019-01-01,1.0,1,51,2018,LED/전구/조명201851
16,LED/전구/조명,2019-01-02,10.0,1,51,2018,LED/전구/조명201851
17,LED/전구/조명,2019-01-03,10.0,1,51,2018,LED/전구/조명201851
18,LED/전구/조명,2019-01-04,2.0,1,51,2018,LED/전구/조명201851
19,LED/전구/조명,2019-01-05,2.0,1,51,2018,LED/전구/조명201851
...,...,...,...,...,...,...,...
56269,화장대/협탁,2019-12-29,NaN,52,50,2019,화장대/협탁201950
56270,화장대/협탁,2019-12-30,NaN,1,51,2019,화장대/협탁201951
56271,화장대/협탁,2019-12-30,NaN,1,51,2019,화장대/협탁201951
56272,화장대/협탁,2019-12-31,NaN,1,51,2019,화장대/협탁201951


## join

In [147]:
# 병합
final2 = pd.merge(final, tmp, how='left', on = 'join_key')
final2

,소분류,날짜,기사건수,실제주,2주전,년도,join_key,2주전_기사건수
0,LED/전구/조명,2019-01-01,1.0,1,51,2018,LED/전구/조명201851,37.0
1,LED/전구/조명,2019-01-02,10.0,1,51,2018,LED/전구/조명201851,37.0
2,LED/전구/조명,2019-01-03,10.0,1,51,2018,LED/전구/조명201851,37.0
3,LED/전구/조명,2019-01-04,2.0,1,51,2018,LED/전구/조명201851,37.0
4,LED/전구/조명,2019-01-05,2.0,1,51,2018,LED/전구/조명201851,37.0
...,...,...,...,...,...,...,...,...
54064,화장대/협탁,2019-12-29,NaN,52,50,2019,화장대/협탁201950,61.0
54065,화장대/협탁,2019-12-30,NaN,1,51,2019,화장대/협탁201951,67.0
54066,화장대/협탁,2019-12-30,NaN,1,51,2019,화장대/협탁201951,67.0
54067,화장대/협탁,2019-12-31,NaN,1,51,2019,화장대/협탁201951,67.0


In [148]:
# 51주 확인
final2[final2['소분류']=='TV']

,소분류,날짜,기사건수,실제주,2주전,년도,join_key,2주전_기사건수
365,TV,2019-01-01,1481.0,1,51,2018,TV201851,14412.0
366,TV,2019-01-02,2393.0,1,51,2018,TV201851,14412.0
367,TV,2019-01-03,2625.0,1,51,2018,TV201851,14412.0
368,TV,2019-01-04,2361.0,1,51,2018,TV201851,14412.0
369,TV,2019-01-05,820.0,1,51,2018,TV201851,14412.0
...,...,...,...,...,...,...,...,...
725,TV,2019-12-27,NaN,52,50,2019,TV201950,14884.0
726,TV,2019-12-28,NaN,52,50,2019,TV201950,14884.0
727,TV,2019-12-29,NaN,52,50,2019,TV201950,14884.0
728,TV,2019-12-30,NaN,1,51,2019,TV201951,12515.0


In [149]:
# 51주 확인
final2[final2['소분류']=='에어컨']

,소분류,날짜,기사건수,실제주,2주전,년도,join_key,2주전_기사건수
31241,에어컨,2019-01-01,27.0,1,51,2018,에어컨201851,230.0
31242,에어컨,2019-01-02,63.0,1,51,2018,에어컨201851,230.0
31243,에어컨,2019-01-03,44.0,1,51,2018,에어컨201851,230.0
31244,에어컨,2019-01-04,33.0,1,51,2018,에어컨201851,230.0
31245,에어컨,2019-01-05,11.0,1,51,2018,에어컨201851,230.0
...,...,...,...,...,...,...,...,...
31601,에어컨,2019-12-27,NaN,52,50,2019,에어컨201950,429.0
31602,에어컨,2019-12-28,NaN,52,50,2019,에어컨201950,429.0
31603,에어컨,2019-12-29,NaN,52,50,2019,에어컨201950,429.0
31604,에어컨,2019-12-30,NaN,1,51,2019,에어컨201951,408.0


In [150]:
# 2주전 기사건수 int type으로 변경
final2['2주전_기사건수'] = final2['2주전_기사건수'].astype(int)

In [151]:
final2.drop(['기사건수','년도','join_key'], axis=1, inplace=True)
final2

,소분류,날짜,실제주,2주전,2주전_기사건수
0,LED/전구/조명,2019-01-01,1,51,37
1,LED/전구/조명,2019-01-02,1,51,37
2,LED/전구/조명,2019-01-03,1,51,37
3,LED/전구/조명,2019-01-04,1,51,37
4,LED/전구/조명,2019-01-05,1,51,37
...,...,...,...,...,...
54064,화장대/협탁,2019-12-29,52,50,61
54065,화장대/협탁,2019-12-30,1,51,67
54066,화장대/협탁,2019-12-30,1,51,67
54067,화장대/협탁,2019-12-31,1,51,67


## 기타_상품군 처리

### 상품군별 '기타_상품군' 차지 비중 계산 (중복 제거된 방송수로 계산)

In [152]:
data = pd.read_csv('data/train_data_06.csv')
data

,index,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,브랜드,...,방송회차,무이자/일시불,프라임시간,평균기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),체감온도,비옴
0,0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트,...,1,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
1,1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트,...,1,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
2,2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트,...,2,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
3,3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트,...,2,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
4,4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트,...,3,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,쿠첸,...,2,일시불,0.0,-7.40,0.0,0.00,57.0,4.5,8.5,0
35375,35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,쿠첸,...,3,무이자,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0
35376,35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,쿠첸,...,3,일시불,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0
35377,35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,쿠첸,...,3,무이자,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0


In [153]:
data = data[['방송일시','상품군','소분류','방송회차']].copy()
data

,방송일시,상품군,소분류,방송회차
0,2019-01-01 06:00:00,의류,니트/가디건,1
1,2019-01-01 06:00:00,의류,니트/가디건,1
2,2019-01-01 06:20:00,의류,니트/가디건,2
3,2019-01-01 06:20:00,의류,니트/가디건,2
4,2019-01-01 06:40:00,의류,니트/가디건,3
...,...,...,...,...
35374,2019-12-31 23:40:00,주방,전기밥솥,2
35375,2020-01-01 00:00:00,주방,전기밥솥,3
35376,2020-01-01 00:00:00,주방,전기밥솥,3
35377,2020-01-01 00:00:00,주방,전기밥솥,3


In [154]:
# 중복제거
data.drop_duplicates(inplace=True)
data

,방송일시,상품군,소분류,방송회차
0,2019-01-01 06:00:00,의류,니트/가디건,1
2,2019-01-01 06:20:00,의류,니트/가디건,2
4,2019-01-01 06:40:00,의류,니트/가디건,3
6,2019-01-01 07:00:00,속옷,브래지어,1
7,2019-01-01 07:20:00,속옷,브래지어,2
...,...,...,...,...
35363,2019-12-31 21:40:00,가전,냉장고,2
35365,2019-12-31 22:00:00,가전,냉장고,3
35367,2019-12-31 23:20:00,주방,전기밥솥,1
35371,2019-12-31 23:40:00,주방,전기밥솥,2


In [155]:
# 중복제거된 상품군별 전체 방송 수
data2 = pd.DataFrame(data['상품군'].value_counts()).reset_index()
data2

,index,상품군
0,농수축,3709
1,의류,3561
2,속옷,3447
3,주방,3305
4,잡화,2062
5,가전,1680
6,생활용품,1481
7,이미용,1270
8,건강기능,648
9,가구,615


In [156]:
# 기타_상품군이 있는 상품군
data[data['소분류'].str.contains('기타')]['상품군'].unique()

array(['건강기능'], dtype=object)

In [157]:
# 기타_상품군만 있는 data
data_etc = data[data['소분류'].str.contains('기타')]
data_etc

,방송일시,상품군,소분류,방송회차
432,2019-01-06 06:00:00,건강기능,기타 건강기능,1
433,2019-01-06 06:20:00,건강기능,기타 건강기능,2
434,2019-01-06 06:40:00,건강기능,기타 건강기능,3
7224,2019-03-19 01:00:00,건강기능,기타 건강기능,1
7225,2019-03-19 01:20:00,건강기능,기타 건강기능,2
7226,2019-03-19 01:40:00,건강기능,기타 건강기능,3


In [158]:
data_etc = pd.DataFrame(data_etc['상품군'].value_counts()).reset_index()
data_etc

,index,상품군
0,건강기능,6


In [159]:
data3 = pd.merge(data2, data_etc, on='index', how='left')
data3

,index,상품군_x,상품군_y
0,농수축,3709,NaN
1,의류,3561,NaN
2,속옷,3447,NaN
3,주방,3305,NaN
4,잡화,2062,NaN
5,가전,1680,NaN
6,생활용품,1481,NaN
7,이미용,1270,NaN
8,건강기능,648,6.0
9,가구,615,NaN


In [160]:
data3.columns = ['상품군','전체개수','기타개수']
data3

,상품군,전체개수,기타개수
0,농수축,3709,NaN
1,의류,3561,NaN
2,속옷,3447,NaN
3,주방,3305,NaN
4,잡화,2062,NaN
5,가전,1680,NaN
6,생활용품,1481,NaN
7,이미용,1270,NaN
8,건강기능,648,6.0
9,가구,615,NaN


In [161]:
data3['기타비중'] = data3['기타개수']/data3['전체개수']
data3

,상품군,전체개수,기타개수,기타비중
0,농수축,3709,NaN,NaN
1,의류,3561,NaN,NaN
2,속옷,3447,NaN,NaN
3,주방,3305,NaN,NaN
4,잡화,2062,NaN,NaN
5,가전,1680,NaN,NaN
6,생활용품,1481,NaN,NaN
7,이미용,1270,NaN,NaN
8,건강기능,648,6.0,0.009259
9,가구,615,NaN,NaN


### 상품군별 '기타 건강기능' 뉴스 건수 행 추가

In [162]:
data = pd.read_csv('data/train_data_06.csv')
data

,index,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,브랜드,...,방송회차,무이자/일시불,프라임시간,평균기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),체감온도,비옴
0,0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트,...,1,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
1,1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트,...,1,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
2,2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트,...,2,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
3,3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트,...,2,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
4,4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트,...,3,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,쿠첸,...,2,일시불,0.0,-7.40,0.0,0.00,57.0,4.5,8.5,0
35375,35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,쿠첸,...,3,무이자,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0
35376,35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,쿠첸,...,3,일시불,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0
35377,35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,쿠첸,...,3,무이자,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0


In [163]:
data.drop(['index'], axis=1, inplace=True)
data

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,브랜드,중분류,...,방송회차,무이자/일시불,프라임시간,평균기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),체감온도,비옴
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트,남성의류,...,1,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트,여성의류,...,1,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트,남성의류,...,2,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트,여성의류,...,2,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트,남성의류,...,3,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,쿠첸,주방가전,...,2,일시불,0.0,-7.40,0.0,0.00,57.0,4.5,8.5,0
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,쿠첸,주방가전,...,3,무이자,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,쿠첸,주방가전,...,3,일시불,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,쿠첸,주방가전,...,3,무이자,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0


In [164]:
data_news_cnt = final2.copy()
data_news_cnt

,소분류,날짜,실제주,2주전,2주전_기사건수
0,LED/전구/조명,2019-01-01,1,51,37
1,LED/전구/조명,2019-01-02,1,51,37
2,LED/전구/조명,2019-01-03,1,51,37
3,LED/전구/조명,2019-01-04,1,51,37
4,LED/전구/조명,2019-01-05,1,51,37
...,...,...,...,...,...
54064,화장대/협탁,2019-12-29,52,50,61
54065,화장대/협탁,2019-12-30,1,51,67
54066,화장대/협탁,2019-12-30,1,51,67
54067,화장대/협탁,2019-12-31,1,51,67


In [165]:
tmp = pd.read_excel('data/이슈크롤링_상품군&소분류 매칭용.xlsx')
tmp

,상품군,소분류
0,생활용품,LED/전구/조명
1,가전,TV
2,가구,TV거실장
3,주방,가스/전기레인지
4,의류,가죽/모피/무스탕
...,...,...
148,이미용,헤어스타일링
149,이미용,헤어케어
150,건강기능,혈행건강
151,건강기능,홍삼


In [166]:
# data_news_cnt에 매칭할 소분류별 '상품군' df 생성
tmp = tmp[['상품군','소분류']].copy()
tmp.drop_duplicates(inplace=True)
tmp

,상품군,소분류
0,생활용품,LED/전구/조명
1,가전,TV
2,가구,TV거실장
3,주방,가스/전기레인지
4,의류,가죽/모피/무스탕
...,...,...
148,이미용,헤어스타일링
149,이미용,헤어케어
150,건강기능,혈행건강
151,건강기능,홍삼


In [167]:
# data_news_cnt에 '상품군' 컬럼 추가 - join을 통해
data_news_cnt = pd.merge(data_news_cnt, tmp, how = 'left', on = '소분류')
data_news_cnt

,소분류,날짜,실제주,2주전,2주전_기사건수,상품군
0,LED/전구/조명,2019-01-01,1,51,37,생활용품
1,LED/전구/조명,2019-01-02,1,51,37,생활용품
2,LED/전구/조명,2019-01-03,1,51,37,생활용품
3,LED/전구/조명,2019-01-04,1,51,37,생활용품
4,LED/전구/조명,2019-01-05,1,51,37,생활용품
...,...,...,...,...,...,...
55898,화장대/협탁,2019-12-29,52,50,61,가구
55899,화장대/협탁,2019-12-30,1,51,67,가구
55900,화장대/협탁,2019-12-30,1,51,67,가구
55901,화장대/협탁,2019-12-31,1,51,67,가구


In [168]:
data_news_cnt.isnull().sum()

소분류         0
날짜          0
실제주         0
2주전         0
2주전_기사건수    0
상품군         0
dtype: int64

In [169]:
data_news_cnt2 = data_news_cnt.groupby(['상품군','날짜'])[['2주전_기사건수']].sum().reset_index()
data_news_cnt2

,상품군,날짜,2주전_기사건수
0,가구,2019-01-01,1393
1,가구,2019-01-02,1393
2,가구,2019-01-03,1393
3,가구,2019-01-04,1393
4,가구,2019-01-05,1393
...,...,...,...
4010,침구,2019-12-27,1208
4011,침구,2019-12-28,1208
4012,침구,2019-12-29,1208
4013,침구,2019-12-30,850


In [170]:
etc_list = data[data['소분류'].str.contains('기타')]['소분류'].unique()
etc_list

array(['기타 건강기능'], dtype=object)

In [171]:
day_list = pd.date_range(start='1/1/19', end = '12/31/19')   # 월/일/년
day_list

DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
               '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08',
               '2019-01-09', '2019-01-10',
               ...
               '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25',
               '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', length=365, freq='D')

In [172]:
type(day_list), len(day_list)

(pandas.core.indexes.datetimes.DatetimeIndex, 365)

In [173]:
# 날짜만 뽑아서, string type으로 변경
day_list = list(map(str, day_list.date))
day_list

['2019-01-01',
 '2019-01-02',
 '2019-01-03',
 '2019-01-04',
 '2019-01-05',
 '2019-01-06',
 '2019-01-07',
 '2019-01-08',
 '2019-01-09',
 '2019-01-10',
 '2019-01-11',
 '2019-01-12',
 '2019-01-13',
 '2019-01-14',
 '2019-01-15',
 '2019-01-16',
 '2019-01-17',
 '2019-01-18',
 '2019-01-19',
 '2019-01-20',
 '2019-01-21',
 '2019-01-22',
 '2019-01-23',
 '2019-01-24',
 '2019-01-25',
 '2019-01-26',
 '2019-01-27',
 '2019-01-28',
 '2019-01-29',
 '2019-01-30',
 '2019-01-31',
 '2019-02-01',
 '2019-02-02',
 '2019-02-03',
 '2019-02-04',
 '2019-02-05',
 '2019-02-06',
 '2019-02-07',
 '2019-02-08',
 '2019-02-09',
 '2019-02-10',
 '2019-02-11',
 '2019-02-12',
 '2019-02-13',
 '2019-02-14',
 '2019-02-15',
 '2019-02-16',
 '2019-02-17',
 '2019-02-18',
 '2019-02-19',
 '2019-02-20',
 '2019-02-21',
 '2019-02-22',
 '2019-02-23',
 '2019-02-24',
 '2019-02-25',
 '2019-02-26',
 '2019-02-27',
 '2019-02-28',
 '2019-03-01',
 '2019-03-02',
 '2019-03-03',
 '2019-03-04',
 '2019-03-05',
 '2019-03-06',
 '2019-03-07',
 '2019-03-

In [174]:
data3

,상품군,전체개수,기타개수,기타비중
0,농수축,3709,NaN,NaN
1,의류,3561,NaN,NaN
2,속옷,3447,NaN,NaN
3,주방,3305,NaN,NaN
4,잡화,2062,NaN,NaN
5,가전,1680,NaN,NaN
6,생활용품,1481,NaN,NaN
7,이미용,1270,NaN,NaN
8,건강기능,648,6.0,0.009259
9,가구,615,NaN,NaN


In [175]:
etc_list

array(['기타 건강기능'], dtype=object)

In [176]:
etc_result = []
for day in day_list:
    # 해당 날짜 상품군의 2주전_기사건수 * 기타_상품군 비중 = 기타 2주전 기사건수
    item = '건강기능'  # 기타 건강기능의 상품군
    idx1 = data_news_cnt2[(data_news_cnt2['상품군'] == item) & (data_news_cnt2['날짜'] == day)].index[0]
    etc_cnt =  round(data_news_cnt2.loc[idx1,'2주전_기사건수'] * 0.009259)
    etc_result.append(['기타 건강기능', day, etc_cnt])

In [177]:
etc_df = pd.DataFrame(etc_result, columns = ['소분류','날짜','2주전_기사건수'])
etc_df

,소분류,날짜,2주전_기사건수
0,기타 건강기능,2019-01-01,29
1,기타 건강기능,2019-01-02,29
2,기타 건강기능,2019-01-03,29
3,기타 건강기능,2019-01-04,29
4,기타 건강기능,2019-01-05,29
...,...,...,...
360,기타 건강기능,2019-12-27,50
361,기타 건강기능,2019-12-28,50
362,기타 건강기능,2019-12-29,50
363,기타 건강기능,2019-12-30,50


In [178]:
etc_df.isnull().sum()

소분류         0
날짜          0
2주전_기사건수    0
dtype: int64

In [179]:
# 기타_상품군 행추가 전, 필요한 컬럼만 남기기
data_news_cnt3 = data_news_cnt[['소분류','날짜','2주전_기사건수']].copy()
data_news_cnt3

,소분류,날짜,2주전_기사건수
0,LED/전구/조명,2019-01-01,37
1,LED/전구/조명,2019-01-02,37
2,LED/전구/조명,2019-01-03,37
3,LED/전구/조명,2019-01-04,37
4,LED/전구/조명,2019-01-05,37
...,...,...,...
55898,화장대/협탁,2019-12-29,61
55899,화장대/협탁,2019-12-30,67
55900,화장대/협탁,2019-12-30,67
55901,화장대/협탁,2019-12-31,67


In [180]:
data_news_cnt3['날짜'] = data_news_cnt3['날짜'].astype(str)

In [181]:
# 최종
data_news_cnt4 = pd.concat([data_news_cnt3, etc_df])
data_news_cnt4

,소분류,날짜,2주전_기사건수
0,LED/전구/조명,2019-01-01,37
1,LED/전구/조명,2019-01-02,37
2,LED/전구/조명,2019-01-03,37
3,LED/전구/조명,2019-01-04,37
4,LED/전구/조명,2019-01-05,37
...,...,...,...
360,기타 건강기능,2019-12-27,50
361,기타 건강기능,2019-12-28,50
362,기타 건강기능,2019-12-29,50
363,기타 건강기능,2019-12-30,50


In [182]:
# 중복제거
data_news_cnt4.drop_duplicates(inplace=True)

In [183]:
data_news_cnt4.shape

(54020, 3)

## join key 생성
- 소분류+날짜

In [184]:
data_news_cnt4['join_key'] = data_news_cnt4['소분류'] + data_news_cnt4['날짜']
data_news_cnt4

,소분류,날짜,2주전_기사건수,join_key
0,LED/전구/조명,2019-01-01,37,LED/전구/조명2019-01-01
1,LED/전구/조명,2019-01-02,37,LED/전구/조명2019-01-02
2,LED/전구/조명,2019-01-03,37,LED/전구/조명2019-01-03
3,LED/전구/조명,2019-01-04,37,LED/전구/조명2019-01-04
4,LED/전구/조명,2019-01-05,37,LED/전구/조명2019-01-05
...,...,...,...,...
360,기타 건강기능,2019-12-27,50,기타 건강기능2019-12-27
361,기타 건강기능,2019-12-28,50,기타 건강기능2019-12-28
362,기타 건강기능,2019-12-29,50,기타 건강기능2019-12-29
363,기타 건강기능,2019-12-30,50,기타 건강기능2019-12-30


In [185]:
data_news_cnt4 = data_news_cnt4[['join_key','2주전_기사건수']].copy()
data_news_cnt4

,join_key,2주전_기사건수
0,LED/전구/조명2019-01-01,37
1,LED/전구/조명2019-01-02,37
2,LED/전구/조명2019-01-03,37
3,LED/전구/조명2019-01-04,37
4,LED/전구/조명2019-01-05,37
...,...,...
360,기타 건강기능2019-12-27,50
361,기타 건강기능2019-12-28,50
362,기타 건강기능2019-12-29,50
363,기타 건강기능2019-12-30,50


In [186]:
data['join_key'] = data['소분류']+data['방송날짜']
data

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,브랜드,중분류,...,무이자/일시불,프라임시간,평균기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),체감온도,비옴,join_key
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트,남성의류,...,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0,니트/가디건2019-01-01
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트,여성의류,...,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0,니트/가디건2019-01-01
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트,남성의류,...,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0,니트/가디건2019-01-01
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트,여성의류,...,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0,니트/가디건2019-01-01
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트,남성의류,...,NaN,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0,니트/가디건2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,쿠첸,주방가전,...,일시불,0.0,-7.40,0.0,0.00,57.0,4.5,8.5,0,전기밥솥2019-12-31
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,쿠첸,주방가전,...,무이자,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0,전기밥솥2020-01-01
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,쿠첸,주방가전,...,일시불,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0,전기밥솥2020-01-01
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,쿠첸,주방가전,...,무이자,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0,전기밥솥2020-01-01


## join

In [187]:
data2 = pd.merge(data, data_news_cnt4, how='left', on = 'join_key')  # 35379 row
data2

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,브랜드,중분류,...,프라임시간,평균기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),체감온도,비옴,join_key,2주전_기사건수
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트,남성의류,...,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0,니트/가디건2019-01-01,336.0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트,여성의류,...,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0,니트/가디건2019-01-01,336.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트,남성의류,...,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0,니트/가디건2019-01-01,336.0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트,여성의류,...,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0,니트/가디건2019-01-01,336.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트,남성의류,...,0.0,-8.05,0.0,1.00,73.0,3.5,-9.8,0,니트/가디건2019-01-01,336.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,쿠첸,주방가전,...,0.0,-7.40,0.0,0.00,57.0,4.5,8.5,0,전기밥솥2019-12-31,49.0
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,쿠첸,주방가전,...,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0,전기밥솥2020-01-01,NaN
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,쿠첸,주방가전,...,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0,전기밥솥2020-01-01,NaN
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,쿠첸,주방가전,...,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0,전기밥솥2020-01-01,NaN


In [188]:
data2[data2['2주전_기사건수'].isnull()]

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,브랜드,중분류,...,프라임시간,평균기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),체감온도,비옴,join_key,2주전_기사건수
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,쿠첸,주방가전,...,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0,전기밥솥2020-01-01,NaN
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,쿠첸,주방가전,...,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0,전기밥솥2020-01-01,NaN
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,쿠첸,주방가전,...,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0,전기밥솥2020-01-01,NaN
35378,2020-01-01 00:00:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,46608000.0,쿠첸,주방가전,...,0.0,-6.25,0.0,0.75,51.0,9.0,-7.0,0,전기밥솥2020-01-01,NaN


In [189]:
data_news_cnt4[data_news_cnt4['join_key'].str.contains('전기밥솥')]

,join_key,2주전_기사건수
38595,전기밥솥2019-01-01,54
38596,전기밥솥2019-01-02,54
38597,전기밥솥2019-01-03,54
38598,전기밥솥2019-01-04,54
38599,전기밥솥2019-01-05,54
...,...,...
38955,전기밥솥2019-12-27,52
38956,전기밥솥2019-12-28,52
38957,전기밥솥2019-12-29,52
38958,전기밥솥2019-12-30,49


In [190]:
# 전기밥솥2020-01-01의 2주전 기사건수 = 49
data2.loc[data2['join_key'] == '전기밥솥2020-01-01','2주전_기사건수'] = 49

In [191]:
data2.drop(['join_key'], axis=1, inplace=True)

In [192]:
data2.rename(columns = {'2주전_기사건수':'뉴스건수'}, inplace=True)

In [193]:
data2['뉴스건수'] = data2['뉴스건수'].astype(int)
data2.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,브랜드,중분류,...,무이자/일시불,프라임시간,평균기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),체감온도,비옴,뉴스건수
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,테이트,남성의류,...,NaN,0.0,-8.05,0.0,1.0,73.0,3.5,-9.8,0,336
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,테이트,여성의류,...,NaN,0.0,-8.05,0.0,1.0,73.0,3.5,-9.8,0,336
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,테이트,남성의류,...,NaN,0.0,-8.05,0.0,1.0,73.0,3.5,-9.8,0,336
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,테이트,여성의류,...,NaN,0.0,-8.05,0.0,1.0,73.0,3.5,-9.8,0,336
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,테이트,남성의류,...,NaN,0.0,-8.05,0.0,1.0,73.0,3.5,-9.8,0,336


In [194]:
data2.to_csv('data/train_data_08.csv', index=False, encoding='utf-8')

# Test data에 '뉴스건수' 컬럼추가

## 데이터 적용 형태로 변경 (일별 -> 주 단위)

In [48]:
import pandas as pd
data_news_cnt = pd.read_csv('data/소분류별_뉴스건수크롤링(20200518~20200630).csv')
data_news_cnt

,소분류,날짜,기사건수
0,LED/전구/조명,2020-05-18,4
1,LED/전구/조명,2020-05-19,3
2,LED/전구/조명,2020-05-20,8
3,LED/전구/조명,2020-05-21,1
4,LED/전구/조명,2020-05-22,1
...,...,...,...
6463,화장대/협탁,2020-06-26,11
6464,화장대/협탁,2020-06-27,4
6465,화장대/협탁,2020-06-28,8
6466,화장대/협탁,2020-06-29,10


In [49]:
import datetime
# 날짜 type object -> datetime으로 변경
data_news_cnt['날짜'] = pd.to_datetime(data_news_cnt['날짜'])

In [50]:
# 월요일이 한주의 시작 기준
data_news_cnt['실제주'] = data_news_cnt['날짜'].map(lambda x: x.isocalendar()[1])

In [51]:
data_news_cnt

,소분류,날짜,기사건수,실제주
0,LED/전구/조명,2020-05-18,4,21
1,LED/전구/조명,2020-05-19,3,21
2,LED/전구/조명,2020-05-20,8,21
3,LED/전구/조명,2020-05-21,1,21
4,LED/전구/조명,2020-05-22,1,21
...,...,...,...,...
6463,화장대/협탁,2020-06-26,11,26
6464,화장대/협탁,2020-06-27,4,26
6465,화장대/협탁,2020-06-28,8,26
6466,화장대/협탁,2020-06-29,10,27


In [52]:
# 2주전 = 실제주 - 2
data_news_cnt['2주전'] = data_news_cnt['실제주'] - 2
data_news_cnt

,소분류,날짜,기사건수,실제주,2주전
0,LED/전구/조명,2020-05-18,4,21,19
1,LED/전구/조명,2020-05-19,3,21,19
2,LED/전구/조명,2020-05-20,8,21,19
3,LED/전구/조명,2020-05-21,1,21,19
4,LED/전구/조명,2020-05-22,1,21,19
...,...,...,...,...,...
6463,화장대/협탁,2020-06-26,11,26,24
6464,화장대/협탁,2020-06-27,4,26,24
6465,화장대/협탁,2020-06-28,8,26,24
6466,화장대/협탁,2020-06-29,10,27,25


## '2주전_기사건수' 컬럼 추가(join 방식으로)

In [53]:
# 2주전 컬럼에 join할 data
tmp = data_news_cnt.copy()
tmp

,소분류,날짜,기사건수,실제주,2주전
0,LED/전구/조명,2020-05-18,4,21,19
1,LED/전구/조명,2020-05-19,3,21,19
2,LED/전구/조명,2020-05-20,8,21,19
3,LED/전구/조명,2020-05-21,1,21,19
4,LED/전구/조명,2020-05-22,1,21,19
...,...,...,...,...,...
6463,화장대/협탁,2020-06-26,11,26,24
6464,화장대/협탁,2020-06-27,4,26,24
6465,화장대/협탁,2020-06-28,8,26,24
6466,화장대/협탁,2020-06-29,10,27,25


In [54]:
# main data
# 6월 data만 남기기
data_news_cnt = data_news_cnt[data_news_cnt['날짜'] >= '2020-06-01']
data_news_cnt

,소분류,날짜,기사건수,실제주,2주전
14,LED/전구/조명,2020-06-01,10,23,21
15,LED/전구/조명,2020-06-02,5,23,21
16,LED/전구/조명,2020-06-03,6,23,21
17,LED/전구/조명,2020-06-04,10,23,21
18,LED/전구/조명,2020-06-05,5,23,21
...,...,...,...,...,...
6463,화장대/협탁,2020-06-26,11,26,24
6464,화장대/협탁,2020-06-27,4,26,24
6465,화장대/협탁,2020-06-28,8,26,24
6466,화장대/협탁,2020-06-29,10,27,25


In [55]:
tmp.drop(['2주전','날짜'], axis=1, inplace=True)
tmp

,소분류,기사건수,실제주
0,LED/전구/조명,4,21
1,LED/전구/조명,3,21
2,LED/전구/조명,8,21
3,LED/전구/조명,1,21
4,LED/전구/조명,1,21
...,...,...,...
6463,화장대/협탁,11,26
6464,화장대/협탁,4,26
6465,화장대/협탁,8,26
6466,화장대/협탁,10,27


## join key 생성

In [56]:
tmp['실제주'] = tmp['실제주'].astype(str)
data_news_cnt['2주전'] = data_news_cnt['2주전'].astype(str)

C:\Users\14Z960-GR30K\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [57]:
tmp['join_key'] = tmp['소분류'] + tmp['실제주']
tmp

,소분류,기사건수,실제주,join_key
0,LED/전구/조명,4,21,LED/전구/조명21
1,LED/전구/조명,3,21,LED/전구/조명21
2,LED/전구/조명,8,21,LED/전구/조명21
3,LED/전구/조명,1,21,LED/전구/조명21
4,LED/전구/조명,1,21,LED/전구/조명21
...,...,...,...,...
6463,화장대/협탁,11,26,화장대/협탁26
6464,화장대/협탁,4,26,화장대/협탁26
6465,화장대/협탁,8,26,화장대/협탁26
6466,화장대/협탁,10,27,화장대/협탁27


In [58]:
# groupby
tmp = tmp.groupby(['join_key'])[['기사건수']].sum().reset_index()
tmp.columns = ['join_key','2주전_기사건수']
tmp

,join_key,2주전_기사건수
0,LED/전구/조명21,19
1,LED/전구/조명22,54
2,LED/전구/조명23,46
3,LED/전구/조명24,29
4,LED/전구/조명25,29
...,...,...
1024,화장대/협탁23,55
1025,화장대/협탁24,35
1026,화장대/협탁25,50
1027,화장대/협탁26,54


In [59]:
data_news_cnt['join_key'] = data_news_cnt['소분류'] + data_news_cnt['2주전']
data_news_cnt

C:\Users\14Z960-GR30K\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,소분류,날짜,기사건수,실제주,2주전,join_key
14,LED/전구/조명,2020-06-01,10,23,21,LED/전구/조명21
15,LED/전구/조명,2020-06-02,5,23,21,LED/전구/조명21
16,LED/전구/조명,2020-06-03,6,23,21,LED/전구/조명21
17,LED/전구/조명,2020-06-04,10,23,21,LED/전구/조명21
18,LED/전구/조명,2020-06-05,5,23,21,LED/전구/조명21
...,...,...,...,...,...,...
6463,화장대/협탁,2020-06-26,11,26,24,화장대/협탁24
6464,화장대/협탁,2020-06-27,4,26,24,화장대/협탁24
6465,화장대/협탁,2020-06-28,8,26,24,화장대/협탁24
6466,화장대/협탁,2020-06-29,10,27,25,화장대/협탁25


## join

In [60]:
data_news_cnt2 = pd.merge(data_news_cnt, tmp, how='left', on = 'join_key')
data_news_cnt2

,소분류,날짜,기사건수,실제주,2주전,join_key,2주전_기사건수
0,LED/전구/조명,2020-06-01,10,23,21,LED/전구/조명21,19
1,LED/전구/조명,2020-06-02,5,23,21,LED/전구/조명21,19
2,LED/전구/조명,2020-06-03,6,23,21,LED/전구/조명21,19
3,LED/전구/조명,2020-06-04,10,23,21,LED/전구/조명21,19
4,LED/전구/조명,2020-06-05,5,23,21,LED/전구/조명21,19
...,...,...,...,...,...,...,...
4405,화장대/협탁,2020-06-26,11,26,24,화장대/협탁24,35
4406,화장대/협탁,2020-06-27,4,26,24,화장대/협탁24,35
4407,화장대/협탁,2020-06-28,8,26,24,화장대/협탁24,35
4408,화장대/협탁,2020-06-29,10,27,25,화장대/협탁25,50


In [61]:
data_news_cnt2.isnull().sum()

소분류         0
날짜          0
기사건수        0
실제주         0
2주전         0
join_key    0
2주전_기사건수    0
dtype: int64

In [62]:
# 불필요한 컬럼 삭제
data_news_cnt2.drop(['기사건수', '실제주', '2주전', 'join_key'], axis=1, inplace=True)

In [63]:
data_news_cnt2

,소분류,날짜,2주전_기사건수
0,LED/전구/조명,2020-06-01,19
1,LED/전구/조명,2020-06-02,19
2,LED/전구/조명,2020-06-03,19
3,LED/전구/조명,2020-06-04,19
4,LED/전구/조명,2020-06-05,19
...,...,...,...
4405,화장대/협탁,2020-06-26,35
4406,화장대/협탁,2020-06-27,35
4407,화장대/협탁,2020-06-28,35
4408,화장대/협탁,2020-06-29,50


##  test data와 join할 join key 생성

In [64]:
data_news_cnt2['날짜'] = data_news_cnt2['날짜'].astype(str)

In [65]:
data_news_cnt2['join_key'] = data_news_cnt2['소분류'] + data_news_cnt2['날짜']
data_news_cnt2

,소분류,날짜,2주전_기사건수,join_key
0,LED/전구/조명,2020-06-01,19,LED/전구/조명2020-06-01
1,LED/전구/조명,2020-06-02,19,LED/전구/조명2020-06-02
2,LED/전구/조명,2020-06-03,19,LED/전구/조명2020-06-03
3,LED/전구/조명,2020-06-04,19,LED/전구/조명2020-06-04
4,LED/전구/조명,2020-06-05,19,LED/전구/조명2020-06-05
...,...,...,...,...
4405,화장대/협탁,2020-06-26,35,화장대/협탁2020-06-26
4406,화장대/협탁,2020-06-27,35,화장대/협탁2020-06-27
4407,화장대/협탁,2020-06-28,35,화장대/협탁2020-06-28
4408,화장대/협탁,2020-06-29,50,화장대/협탁2020-06-29


In [66]:
# join에 필요한 컬럼만 남기기
data_news_cnt3 = data_news_cnt2[['join_key','2주전_기사건수']].copy()
data_news_cnt3

,join_key,2주전_기사건수
0,LED/전구/조명2020-06-01,19
1,LED/전구/조명2020-06-02,19
2,LED/전구/조명2020-06-03,19
3,LED/전구/조명2020-06-04,19
4,LED/전구/조명2020-06-05,19
...,...,...
4405,화장대/협탁2020-06-26,35
4406,화장대/협탁2020-06-27,35
4407,화장대/협탁2020-06-28,35
4408,화장대/협탁2020-06-29,50


In [96]:
# test_data_06
data = pd.read_csv('data/test_data_06.csv')
data

,index,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,브랜드,...,방송회차,무이자/일시불,프라임시간,평균기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),체감온도,비옴
0,0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,...,1,NaN,0.0,16.70,0.10,2.90,94.5,6.0,16.6,0
1,1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,...,2,NaN,0.0,16.70,0.10,2.90,94.5,6.0,16.6,0
2,2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,...,3,NaN,0.0,16.90,0.00,2.80,90.0,6.0,16.9,0
3,3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,쿠미투니카,...,1,NaN,1.0,16.90,0.00,2.80,90.0,6.0,16.9,0
4,4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,쿠미투니카,...,2,NaN,1.0,16.90,0.00,2.80,90.0,6.0,16.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,...,1,NaN,0.0,18.35,0.25,2.25,91.5,10.0,18.8,0
2887,2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,...,2,NaN,0.0,18.35,0.25,2.25,91.5,10.0,18.8,0
2888,2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,...,3,NaN,0.0,18.50,0.05,2.30,92.5,10.0,19.0,0
2889,2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,아놀드파마,...,1,NaN,0.0,18.50,0.05,2.30,92.5,10.0,19.0,0


In [97]:
data.drop(['index'], axis=1, inplace=True)

In [98]:
# test_data_06
data['join_key'] = data['소분류']+data['방송날짜']
data

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,브랜드,중분류,...,무이자/일시불,프라임시간,평균기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),체감온도,비옴,join_key
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,남성의류,...,NaN,0.0,16.70,0.10,2.90,94.5,6.0,16.6,0,셔츠/남방2020-06-01
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,남성의류,...,NaN,0.0,16.70,0.10,2.90,94.5,6.0,16.6,0,셔츠/남방2020-06-01
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,남성의류,...,NaN,0.0,16.90,0.00,2.80,90.0,6.0,16.9,0,셔츠/남방2020-06-01
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,쿠미투니카,여성속옷,...,NaN,1.0,16.90,0.00,2.80,90.0,6.0,16.9,0,팬티2020-06-01
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,쿠미투니카,여성속옷,...,NaN,1.0,16.90,0.00,2.80,90.0,6.0,16.9,0,팬티2020-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,NaN,...,NaN,0.0,18.35,0.25,2.25,91.5,10.0,18.8,0,NaN
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,NaN,...,NaN,0.0,18.35,0.25,2.25,91.5,10.0,18.8,0,NaN
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,NaN,...,NaN,0.0,18.50,0.05,2.30,92.5,10.0,19.0,0,NaN
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,아놀드파마,여성의류,...,NaN,0.0,18.50,0.05,2.30,92.5,10.0,19.0,0,블라우스/셔츠2020-07-01


## join

In [99]:
data3 = pd.merge(data, data_news_cnt3, how='left', on = 'join_key')  # 2891 row
data3

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,브랜드,중분류,...,프라임시간,평균기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),체감온도,비옴,join_key,2주전_기사건수
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,남성의류,...,0.0,16.70,0.10,2.90,94.5,6.0,16.6,0,셔츠/남방2020-06-01,1857.0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,남성의류,...,0.0,16.70,0.10,2.90,94.5,6.0,16.6,0,셔츠/남방2020-06-01,1857.0
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,남성의류,...,0.0,16.90,0.00,2.80,90.0,6.0,16.9,0,셔츠/남방2020-06-01,1857.0
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,쿠미투니카,여성속옷,...,1.0,16.90,0.00,2.80,90.0,6.0,16.9,0,팬티2020-06-01,75.0
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,쿠미투니카,여성속옷,...,1.0,16.90,0.00,2.80,90.0,6.0,16.9,0,팬티2020-06-01,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,NaN,...,0.0,18.35,0.25,2.25,91.5,10.0,18.8,0,NaN,NaN
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,NaN,...,0.0,18.35,0.25,2.25,91.5,10.0,18.8,0,NaN,NaN
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,NaN,...,0.0,18.50,0.05,2.30,92.5,10.0,19.0,0,NaN,NaN
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,아놀드파마,여성의류,...,0.0,18.50,0.05,2.30,92.5,10.0,19.0,0,블라우스/셔츠2020-07-01,NaN


In [100]:
data3[data3['2주전_기사건수'].isnull()]['join_key'].value_counts()

드로즈2020-07-01        6
블라우스/셔츠2020-07-01    2
Name: join_key, dtype: int64

In [101]:
data_news_cnt3[data_news_cnt3['join_key'].str.contains('블라우스')]

,join_key,2주전_기사건수
1650,블라우스/셔츠2020-06-01,143
1651,블라우스/셔츠2020-06-02,143
1652,블라우스/셔츠2020-06-03,143
1653,블라우스/셔츠2020-06-04,143
1654,블라우스/셔츠2020-06-05,143
1655,블라우스/셔츠2020-06-06,143
1656,블라우스/셔츠2020-06-07,143
1657,블라우스/셔츠2020-06-08,150
1658,블라우스/셔츠2020-06-09,150
1659,블라우스/셔츠2020-06-10,150


In [102]:
# 2020-07-01 -> 2020-06-30 data로 대체 (같은 주라서 가능)
data3.loc[data3['join_key'] == '드로즈2020-07-01','2주전_기사건수'] = 37
data3.loc[data3['join_key'] == '블라우스/셔츠2020-07-01','2주전_기사건수'] = 133

In [103]:
data3[data3['2주전_기사건수'].isnull()]['join_key'].value_counts()

Series([], Name: join_key, dtype: int64)

In [104]:
data3.rename(columns = {'2주전_기사건수':'뉴스건수'}, inplace=True)
data3

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,브랜드,중분류,...,프라임시간,평균기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),체감온도,비옴,join_key,뉴스건수
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,남성의류,...,0.0,16.70,0.10,2.90,94.5,6.0,16.6,0,셔츠/남방2020-06-01,1857.0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,남성의류,...,0.0,16.70,0.10,2.90,94.5,6.0,16.6,0,셔츠/남방2020-06-01,1857.0
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,남성의류,...,0.0,16.90,0.00,2.80,90.0,6.0,16.9,0,셔츠/남방2020-06-01,1857.0
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,쿠미투니카,여성속옷,...,1.0,16.90,0.00,2.80,90.0,6.0,16.9,0,팬티2020-06-01,75.0
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,쿠미투니카,여성속옷,...,1.0,16.90,0.00,2.80,90.0,6.0,16.9,0,팬티2020-06-01,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,NaN,...,0.0,18.35,0.25,2.25,91.5,10.0,18.8,0,NaN,NaN
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,NaN,...,0.0,18.35,0.25,2.25,91.5,10.0,18.8,0,NaN,NaN
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,NaN,...,0.0,18.50,0.05,2.30,92.5,10.0,19.0,0,NaN,NaN
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,아놀드파마,여성의류,...,0.0,18.50,0.05,2.30,92.5,10.0,19.0,0,블라우스/셔츠2020-07-01,133.0


In [105]:
data3.drop(['join_key'], axis=1, inplace=True)
data3

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,브랜드,중분류,...,무이자/일시불,프라임시간,평균기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),체감온도,비옴,뉴스건수
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,남성의류,...,NaN,0.0,16.70,0.10,2.90,94.5,6.0,16.6,0,1857.0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,남성의류,...,NaN,0.0,16.70,0.10,2.90,94.5,6.0,16.6,0,1857.0
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,잭필드,남성의류,...,NaN,0.0,16.90,0.00,2.80,90.0,6.0,16.9,0,1857.0
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,쿠미투니카,여성속옷,...,NaN,1.0,16.90,0.00,2.80,90.0,6.0,16.9,0,75.0
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,쿠미투니카,여성속옷,...,NaN,1.0,16.90,0.00,2.80,90.0,6.0,16.9,0,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,NaN,...,NaN,0.0,18.35,0.25,2.25,91.5,10.0,18.8,0,NaN
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,NaN,...,NaN,0.0,18.35,0.25,2.25,91.5,10.0,18.8,0,NaN
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,NaN,NaN,...,NaN,0.0,18.50,0.05,2.30,92.5,10.0,19.0,0,NaN
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,아놀드파마,여성의류,...,NaN,0.0,18.50,0.05,2.30,92.5,10.0,19.0,0,133.0


In [107]:
data3[data3['상품군'] == '무형']['뉴스건수'].unique()

array([nan])

In [109]:
data3.to_csv('data/test_data_08.csv', index=False, encoding='utf-8')